# Set up

In [1]:
library("tidyverse")
library("janitor")

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.4     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




# Load data

In [2]:
## CYP2D6 consensus haplotype calls
## s3://sg10k-cyp2d6-workspace/call_haplotypes/pilot/consensus_cyrius_aldy_stellarpgx.csv
## note: not shared with nala yet

in_csv = "/data/SG10K-CYP2D6/sandbox/aggregate_stats/consensus_cyrius_aldy_stellarpgx.csv"
calls = read_csv(in_csv, col_types = cols()) %>% clean_names()
calls %>% head

# sanity checks
calls %>% count()

sample_id,cyrius_filter,cyrius_diplotype_original,cyrius_diplotype,aldy_filter,aldy_diplotype_original,aldy_diplotype,stellarpgx_filter,stellarpgx_diplotype_original,stellarpgx_diplotype,consensus_filter,consensus_diplotype
<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
WHB10000,PASS,*10/*2,*2/*10,NA,*2/*13,*2/*13,NA,*10/*2,*2/*10,PASS,*2/*10
WHB10001,None,None,no_call,NA,*36.ALDY+*10/*63,*36+*10/*63,NA,*2/*36+*10,*2/*10+*36,FAIL,no_call
WHB10002,PASS,*2/*39,*2/*39,NA,*2/*39,*2/*39,NA,*2/*39,*2/*39,PASS,*2/*39
WHB10003,None,None,no_call,NA,*1+rs1065852/*10,*1+rs1065852/*10,NA,Possible novel allele or suballele present: interpret with caution; experimental validation and expert review through PharmVar is recommended,possible_novel_allele,FAIL,no_call
WHB10005,PASS,*10/*36+*10,*10/*10+*36,NA,*10/*36.ALDY+*10,*10/*10+*36,NA,*10/*36x2,*10/*36x2,PASS,*10/*10+*36
WHB10006,None,None,no_call,NA,*10+*10+*10+*10+*10+*10+*36.ALDY+*10/*36.ALDY+*10,*10x7+*36/*10+*36,NA,NA,no_call,FAIL,no_call


n
<int>
90


In [3]:
## ethnicity information
## s3://sg10k-cyp2d6/metadata/SG10K-CYP2D6.sample_list.20210722.csv

in_csv = "/data/SG10K-CYP2D6/s3/sg10k-cyp2d6/metadata/SG10K-CYP2D6.sample_list.20210722.csv"
metadata = read_csv(in_csv, col_types = cols()) %>% clean_names()
metadata %>% head

## sanity checks
## we expect 60 samples at 30X
metadata %>% 
filter(sequencing_depth == "30x") %>% 
filter(npm_research_id %in% calls$sample_id) %>% 
count()

npm_research_id,multiplex_pool_id,supplier_id,site_supplying_sample,supplied_gender,genetic_sex,self_reported_ethnicity,genetic_ancestry,extraction_kit,library_prep_kit,sequencing_depth,estimate_of_sequence_coverage,qc_pass_in_r5_3,industry_consent
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
WHB3854,MUX8960,020-66085-011,GUSTO_Kids,M,male,Chinese,C,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,30.22588,TRUE,TRUE
WHB3855,MUX8960,010-22048,GUSTO_Kids,F,female,Malay,M,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,36.72887,TRUE,TRUE
WHB3856,MUX8960,020-04240,GUSTO_Kids,F,female,Chinese,C,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,22.70551,TRUE,TRUE
WHB3857,MUX8960,020-66123,GUSTO_Kids,M,male,Indian,I,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,31.11992,TRUE,TRUE
WHB3858,MUX8960,010-20267,GUSTO_Kids,F,female,Malay,M,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,26.51214,TRUE,TRUE
WHB3859,MUX8961,010-21795,GUSTO_Kids,F,female,Malay,M,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,29.33307,TRUE,TRUE


n
<int>
60


In [4]:
## merge
a = calls %>% 
select(sample_id, consensus_filter, consensus_diplotype)

b = metadata %>% 
rename(sample_id = npm_research_id)

annotated_calls = merge(a, b, by = "sample_id")
annotated_calls %>% head()
annotated_calls %>%
group_by(consensus_filter) %>% 
count()

,sample_id,consensus_filter,consensus_diplotype,multiplex_pool_id,supplier_id,site_supplying_sample,supplied_gender,genetic_sex,self_reported_ethnicity,genetic_ancestry,extraction_kit,library_prep_kit,sequencing_depth,estimate_of_sequence_coverage,qc_pass_in_r5_3,industry_consent
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>
1,WHB3854,PASS,*1/*10+*36,MUX8960,020-66085-011,GUSTO_Kids,M,male,Chinese,C,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,30.22588,TRUE,TRUE
2,WHB3855,PASS,*1/*10,MUX8960,010-22048,GUSTO_Kids,F,female,Malay,M,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,36.72887,TRUE,TRUE
3,WHB3856,PASS,*10+*36/*10+*36,MUX8960,020-04240,GUSTO_Kids,F,female,Chinese,C,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,22.70551,TRUE,TRUE
4,WHB3857,PASS,*1/*2,MUX8960,020-66123,GUSTO_Kids,M,male,Indian,I,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,31.11992,TRUE,TRUE
5,WHB3858,PASS,*4/*10,MUX8960,010-20267,GUSTO_Kids,F,female,Malay,M,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,26.51214,TRUE,TRUE
6,WHB3859,PASS,*1/*1,MUX8961,010-21795,GUSTO_Kids,F,female,Malay,M,QIAsymphony DSP DNA Midi Kit,NEBNext UltraII DNA library Prep Kit for Illumina,30x,29.33307,TRUE,TRUE


consensus_filter,n
<chr>,<int>
FAIL,8
PASS,52


# Calculate aggregate stats

In [5]:
## Note: using data-derived genetic ancestry, as oppposed to self-reported ethnicity
## Need to account for cases where that info is missing, if any (TODO)

In [6]:
## diplotype-level

## prepare data table
annotated_calls_pass = annotated_calls %>% 
filter(consensus_filter == "PASS")

## calculate stats (count + freq)
TOTAL_DIPLOTYPES = annotated_calls_pass %>% count() %>% unlist()
TOTAL_DIPLOTYPES

aggregate_stats_diplotypes = annotated_calls_pass %>% 
group_by(consensus_diplotype, genetic_ancestry) %>% 
count() %>% 
spread(key = genetic_ancestry, value = n) %>% 
rename(n_chinese = C) %>% 
rename(n_indian = I) %>% 
rename(n_malay = M) %>% 
mutate(n_all = sum(n_chinese, n_indian, n_malay, na.rm = TRUE)) %>% 
mutate(freq_chinese = n_chinese / TOTAL_DIPLOTYPES) %>% 
mutate(freq_indian = n_indian / TOTAL_DIPLOTYPES) %>% 
mutate(freq_malay = n_malay / TOTAL_DIPLOTYPES) %>% 
mutate(freq_all = n_all / TOTAL_DIPLOTYPES) %>% 
ungroup()

aggregate_stats_diplotypes %>% head

## sanity checks
aggregate_stats_diplotypes %>% select(n_all) %>% sum()
aggregate_stats_diplotypes %>% select(freq_all) %>% sum()

## save output
out_csv = "/data/SG10K-CYP2D6/s3/sg10k-cyp2d6/pilot/aggregate_stats_diplotypes.csv"
write_csv(aggregate_stats_diplotypes, file = out_csv)


n 
52

consensus_diplotype,n_chinese,n_indian,n_malay,n_all,freq_chinese,freq_indian,freq_malay,freq_all
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
*1/*1,NA,2,1,3,NA,0.03846154,0.01923077,0.05769231
*1/*10,NA,NA,3,3,NA,NA,0.05769231,0.05769231
*1/*10+*36,2,NA,3,5,0.03846154,NA,0.05769231,0.09615385
*1/*10+*36x2,NA,NA,1,1,NA,NA,0.01923077,0.01923077
*1/*1x2,NA,1,NA,1,NA,0.01923077,NA,0.01923077
*1/*2,1,5,2,8,0.01923077,0.09615385,0.03846154,0.15384615


[1] 52

[1] 1

In [7]:
## haplotype-level

## prepare data table
annotated_calls_pass = annotated_calls %>% 
filter(consensus_filter == "PASS") %>% 
separate_rows(consensus_diplotype, sep = "/") %>% 
rename(consensus_haplotype = consensus_diplotype) %>% 
select(sample_id, consensus_haplotype, genetic_ancestry)

# annotated_calls_pass %>% head

## calculate stats (count + freq)
TOTAL_HAPLOTYPES = annotated_calls_pass %>% count() %>% unlist()
TOTAL_HAPLOTYPES

aggregate_stats_haplotypes = annotated_calls_pass %>% 
group_by(consensus_haplotype, genetic_ancestry) %>% 
count() %>% 
spread(key = genetic_ancestry, value = n) %>% 
rename(n_chinese = C) %>% 
rename(n_indian = I) %>% 
rename(n_malay = M) %>% 
mutate(n_all = sum(n_chinese, n_indian, n_malay, na.rm = TRUE)) %>% 
mutate(freq_chinese = n_chinese / TOTAL_HAPLOTYPES) %>% 
mutate(freq_indian = n_indian / TOTAL_HAPLOTYPES) %>% 
mutate(freq_malay = n_malay / TOTAL_HAPLOTYPES) %>% 
mutate(freq_all = n_all / TOTAL_HAPLOTYPES) %>% 
ungroup()

aggregate_stats_haplotypes %>% head

## sanity checks
aggregate_stats_haplotypes %>% select(n_all) %>% sum() ## 2*N samples
aggregate_stats_haplotypes %>% select(freq_all) %>% sum()

## save output
out_csv = "/data/SG10K-CYP2D6/s3/sg10k-cyp2d6/pilot/aggregate_stats_haplotypes.csv"
write_csv(aggregate_stats_haplotypes, file = out_csv)


n 
104

consensus_haplotype,n_chinese,n_indian,n_malay,n_all,freq_chinese,freq_indian,freq_malay,freq_all
<chr>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
*1,5,14,13,32,0.048076923,0.134615385,0.125000000,0.307692308
*10,3,NA,9,12,0.028846154,NA,0.086538462,0.115384615
*10+*36,10,NA,7,17,0.096153846,NA,0.067307692,0.163461538
*10+*36x2,1,NA,1,2,0.009615385,NA,0.009615385,0.019230769
*112,NA,1,NA,1,NA,0.009615385,NA,0.009615385
*1x2,NA,2,NA,2,NA,0.019230769,NA,0.019230769


[1] 104

[1] 1

# EOF